# Load Ni-Mo data

In [1]:
from pymatgen import Structure
from monty.serialization import loadfn

data = loadfn('data.json')
train_structures = [d['structure'] for d in data]
train_energies = [d['outputs']['energy'] for d in data]
train_forces = [d['outputs']['forces'] for d in data]

# Setup the initial weights for training (If not, the weights for energy and force will be both equal to 1)

In [2]:
import numpy as np
from maml.utils import pool_from, convert_docs

train_pool = pool_from(train_structures, train_energies, train_forces)
_, df = convert_docs(train_pool)

weights = np.ones(len(df['dtype']), )

# set the weights for energy equal to 100
weights[df['dtype'] == 'energy'] = 100

# Set up the SNAP and train

In [3]:
from maml.base import SKLModel
from maml.describers import BispectrumCoefficients
from sklearn.linear_model import LinearRegression
from maml.apps.pes import SNAPotential

element_profile = {'Mo': {'r': 5.0, 'w': 1}, 'Ni': {'r': 5.0, 'w': 1}}
describer = BispectrumCoefficients(rcutfac=0.5, twojmax=6, element_profile=element_profile, 
                                   quadratic=False, pot_fit=True)
model = SKLModel(describer=describer, model=LinearRegression())
snap = SNAPotential(model=model)
snap.train(train_structures, train_energies, train_forces, sample_weight=weights)

# Predict the energies, forces of training data

In [4]:
df_orig, df_predict = snap.evaluate(test_structures=train_structures, 
                                    test_energies=train_energies,
                                    test_forces=train_forces, test_stresses=None)

In [5]:
df_predict[df_predict['dtype'] == 'energy']

,y_orig,n,dtype
0,-1022.370440,144.0,energy
433,-1021.289063,144.0,energy
866,-968.089277,144.0,energy
1299,-974.584719,144.0,energy
1732,-968.786928,144.0,energy
2165,-1008.753287,144.0,energy
2598,-1020.542738,144.0,energy
3031,-958.846064,144.0,energy
3464,-1008.594962,144.0,energy
3897,-972.498568,144.0,energy


# Lattice constant, Elastic constant
### Large error due to limited training data -- 10 structures

In [ ]:
from pymatgen.core import Lattice
Ni = Structure.from_spacegroup(sg='Fm-3m', species=['Ni'], lattice=Lattice.cubic(3.51), coords=[[0, 0, 0]])
Mo = Structure.from_spacegroup(sg='Im-3m', species=['Mo'], lattice=Lattice.cubic(3.17), coords=[[0, 0, 0]])

In [10]:
from maml.apps.pes import LatticeConstant

Ni = Structure.from_spacegroup(sg='Fm-3m', species=['Ni'], lattice=Lattice.cubic(3.51), coords=[[0, 0, 0]])
lc_calculator = LatticeConstant(ff_settings=snap)
a, b, c = lc_calculator.calculate([Ni])[0]
print('Ni', 'Lattice a: {}, Lattice b: {}, Lattice c: {}'.format(a, b, c))

Ni Lattice a: 3.54568992127133, Lattice b: 3.54568992127133, Lattice c: 3.54568992127133


In [12]:
lc_calculator = LatticeConstant(ff_settings=snap)
a, b, c = lc_calculator.calculate([Mo])[0]
print('Mo', 'Lattice a: {}, Lattice b: {}, Lattice c: {}'.format(a, b, c))

Mo Lattice a: 2.96446667369633, Lattice b: 2.96446667369633, Lattice c: 2.96446667369633


In [6]:
from maml.apps.pes import ElasticConstant

Ni_ec_calculator = ElasticConstant(ff_settings=snap)
Ni_C11, Ni_C12, Ni_C44, _ = Ni_ec_calculator.calculate([Ni])[0]
print('Ni', ' C11: ', Ni_C11, 'C12: ', Ni_C12, 'C44: ', Ni_C44)

Ni  C11:  159.82034045317 C12:  236.570065272579 C44:  55.2780305678332


In [8]:
Mo_ec_calculator = ElasticConstant(ff_settings=snap)
Mo_C11, Mo_C12, Mo_C44, _ = Mo_ec_calculator.calculate([Mo])[0]
print('Mo', ' C11: ', Mo_C11, 'C12: ', Mo_C12, 'C44: ', Mo_C44)

Mo  C11:  -0.999219759978588 C12:  108.341414925053 C44:  5.06105663592628


# Load from parameters files

In [14]:
snap_loaded = SNAPotential.from_config(param_file='SNAPotential.snapparam', coeff_file='SNAPotential.snapcoeff')

# Energy, force, stress prediction

In [15]:
from maml.apps.pes import EnergyForceStress

efs_calculator = EnergyForceStress(ff_settings=snap_loaded)
energy, forces, stresses = efs_calculator.calculate([train_structures[0]])[0]

print('energy: {}'.format(energy))
print('forces: \n', forces)
print('stresses: ', stresses)

energy: -1022.37044072151
forces: 
 [[-1.12509e-01  6.06865e-01  2.25153e-02]
 [ 6.24201e-01 -4.05099e-01 -8.10128e-01]
 [-4.23769e-01 -5.67866e-02  5.32287e-01]
 [ 1.74322e-01  4.02397e-01 -3.64179e-01]
 [-5.84530e-01 -1.17059e-01 -1.51485e+00]
 [ 4.67829e-01  7.84477e-02 -3.56594e-01]
 [-7.62658e-01  2.06331e-01  2.27666e-01]
 [-7.54536e-02  4.03482e-02  7.51472e-01]
 [-1.27947e+00 -4.80994e-01  2.42017e-01]
 [ 7.44870e-01 -3.07897e-01 -5.43406e-01]
 [ 5.97091e-02 -1.98807e-01 -1.41200e-02]
 [-8.52300e-01  7.18157e-02 -5.73846e-01]
 [-2.35186e-02 -1.05202e-01  6.04151e-01]
 [-2.02694e-01 -6.82459e-01 -9.84649e-02]
 [ 2.34810e-01 -8.01765e-01 -1.29139e+00]
 [-6.39804e-01 -5.74294e-01  7.74981e-02]
 [-2.79066e-01 -1.06464e-01 -2.14629e-01]
 [-2.88254e-01 -8.20037e-02 -5.44820e-01]
 [ 3.54259e-02 -2.75387e-01  1.16154e+00]
 [-4.38452e-01  1.28732e-01 -4.08173e-01]
 [-7.55223e-01  6.57518e-01  4.13182e-01]
 [-2.47282e-01 -5.77589e-02  1.83189e-01]
 [-3.24134e-01  1.51230e+00  7.10574e-01

# Load Ni-Mo optimized parameters
# Citation: Phys.Rev. B2018,98, 094104

In [16]:
from maml.apps.pes import SNAPotential
from maml.apps.pes import ElasticConstant

snap_prb = SNAPotential.from_config(param_file='Ni_Mo.snapparam', coeff_file='Ni_Mo.snapcoeff')

Ni_ec_calculator = ElasticConstant(ff_settings=snap_prb, lattice='fcc', alat=3.508, atom_type='Ni')
Ni_C11, Ni_C12, Ni_C44, _ = Ni_ec_calculator.calculate()
print('Ni', ' C11: ', Ni_C11, 'C12: ', Ni_C12, 'C44: ', Ni_C44)

Ni  C11:  269.480695900637 C12:  149.622413475444 C44:  135.35839213835


In [17]:
Mo_ec_calculator = ElasticConstant(ff_settings=snap_prb, lattice='bcc', alat=3.168, atom_type='Mo')
Mo_C11, Mo_C12, Mo_C44, _ = Mo_ec_calculator.calculate()
print('Mo', ' C11: ', Mo_C11, 'C12: ', Mo_C12, 'C44: ', Mo_C44)

Mo  C11:  475.137183237399 C12:  162.579135041946 C44:  111.454540936551
